<a href="https://colab.research.google.com/github/abhay43/ML_Code/blob/master/GAN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import fashion_mnist, mnist
((images_train, _), (_, _)) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [ ]:
#normalise the images (-1, 1)
#(0, 255) -> subtract 127.5 fro. this -> (-127.5, 127.5) -> dividing by 127.5 -> (-1, 1)

randomDim = 100 #z

images_train = (np.array(images_train).astype(np.float32) - 127.5)/127.5
images_train = images_train.reshape(60000, 784)

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers

In [ ]:
#Input_shape = 100, output_shape = 784
#kernel_initializer=initializers.RandomNormal(stddev=0.02)= weights eintilization
#100 -> 256 -> 512 -> 1024 -> 784
adam = Adam(lr=0.0002, beta_1=0.5)

generator = Sequential()
generator.add(Dense(256, input_dim=randomDim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
generator.add(LeakyReLU(0.2))
generator.add(Dense(512))
generator.add(LeakyReLU(0.2))
generator.add(Dense(1024))
generator.add(LeakyReLU(0.2))
generator.add(Dense(784, activation='tanh'))
generator.compile(loss='binary_crossentropy', optimizer=adam)

In [ ]:
generator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               25856     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 784)               8

In [ ]:
#784 -> 1024 -> 512 -> 256 -> 1

discriminator = Sequential()
discriminator.add(Dense(1024, input_dim=784, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(512))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer=adam)

In [ ]:
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 1024)              803840    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)              

In [ ]:
discriminator.trainable = False
ganInput = Input(shape=(randomDim,)) #100
x = generator(ganInput)  #784 G(z)
ganOutput = discriminator(x) #1
gan = Model(inputs=ganInput, outputs=ganOutput)
gan.compile(loss='binary_crossentropy', optimizer=adam)

In [ ]:
gan.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
sequential (Sequential)      (None, 784)               1486352   
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 1460225   
Total params: 2,946,577
Trainable params: 1,486,352
Non-trainable params: 1,460,225
_________________________________________________________________


In [ ]:
def plotGeneratedImages(epoch, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, randomDim])
    generatedImages = generator.predict(noise)
    generatedImages = generatedImages.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generatedImages[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_generated_image_epoch_%d.png' % epoch)

In [ ]:
def train(epochs=1, batchSize=128):
    batchCount = int(images_train.shape[0] / batchSize) #60000/128
    print ('Epochs:', epochs)
    print ('Batch size:', batchSize)
    print ('Batches per epoch:', batchCount)

    for e in range(1, epochs+1):
        print ('-'*15, 'Epoch %d' % e, '-'*15)
        for _ in range(batchCount): #128
            # Get a random set of input noise and images
            noise = np.random.normal(0, 1, size=[batchSize, randomDim]) #generating z 128*100
            imageBatch = images_train[np.random.randint(0, images_train.shape[0], size=batchSize)] #128*784

            # Generate fake fashion MNIST images
            generatedImages = generator.predict(noise) #128*784
            # print np.shape(imageBatch), np.shape(generatedImages)
            X = np.concatenate([imageBatch, generatedImages]) #256*784

            # Labels for generated and real data
            yDis = np.zeros(2*batchSize)  #256 size array of all zeros
            # One-sided label smoothing
            yDis[:batchSize] = 0.9  #making real images label = 0.9

            # Train discriminator
            discriminator.trainable = True
            dloss = discriminator.train_on_batch(X, yDis)

            # Train generator
            noise = np.random.normal(0, 1, size=[batchSize, randomDim]) #genaret 128 new noise 128*100
            yGen = np.ones(batchSize) #labels it as one
            discriminator.trainable = False
            gloss = gan.train_on_batch(noise, yGen)

        

        if e == 1 or e % 5 == 0:
            plotGeneratedImages(e)
            



In [ ]:
train(50,100)

Epochs: 50
Batch size: 100
Batches per epoch: 600
--------------- Epoch 1 ---------------
--------------- Epoch 2 ---------------


In [ ]:
ls

In [ ]:
from google.colab import files

In [ ]:
import os

In [ ]:
os.listdir()

In [ ]:
for f in ['gan_generated_image_epoch_10.png',
 'gan_generated_image_epoch_20.png',
 'gan_generated_image_epoch_35.png',
 'gan_generated_image_epoch_25.png',
 'gan_generated_image_epoch_30.png',
 'gan_generated_image_epoch_15.png',
 'gan_generated_image_epoch_5.png',
 'gan_generated_image_epoch_1.png']:
 files.download(f)